## Syntactic sugar

Define a magic to run code via pyodide instead of the python kernel.  Imagine this is wrapped up in a python module.

The first time the magic is used, it does some initialization steps:
 * Load pyodide.js
 * Create a rendering function based on what Iodide uses
 * Add a workaround for matplotlib charts
 
Then each time the magic is used to execute code, it does the following:
 * Creates an IPython HTML output as a placeholder
 * Runs the python code in pyodide
 * Renders whatever value was returned in the placeholder HTML

In [ ]:
from IPython.core.magic import register_line_magic, register_cell_magic
from IPython.display import display, Javascript, HTML
import os

_pyodide_count = 0

@register_cell_magic
def pyodide(line, cell):
    global _pyodide_count
    _pyodide_count += 1
    code = ''
    
    # The first time we run any pyodide code, we need to initialize pyodide itself
    if _pyodide_count == 1:
        # First, load pyodide itself
        # Async script load based on:
        # https://medium.com/@vschroeder/javascript-how-to-execute-code-from-an-asynchronously-loaded-script-although-when-it-is-not-bebcbd6da5ea
        # See also: 
        # https://stackoverflow.com/questions/950087/how-do-i-include-a-javascript-file-in-another-javascript-file
        pyodide_url = os.getenv('PYODIDE_URL', 'https://pyodide-cdn2.iodide.io/v0.15.0/full/pyodide.js')
        code += (
            f"window.pyodideLoaded = new Promise((resolve, reject) => {{\n"
            f"    var script = document.createElement('script');\n"
            f"    script.src = '{pyodide_url}';\n"
            f"    script.async = true;\n"
            f"    script.onload = () => {{\n"
            f"        languagePluginLoader.then(() => {{\n"
            f"            resolve();\n"
            f"        }});\n"
            f"    }};\n"
            f"    document.head.appendChild(script);\n"
            f"}});\n"
        )
        
        # Next, add a function for rendering pyodide outputs
        code += (
            f"window.pyodideRender = (value) => {{\n"
            f"    if (typeof value === 'function' && pyodide._module.PyProxy.isPyProxy(value)) {{\n"
            f"        let div = document.createElement('div');\n"
            f"        div.className = 'rendered_html';\n"
            f"        var element;\n"
            f"        if (value._repr_html_ !== undefined) {{\n"
            f"            let result = value._repr_html_();\n"
            f"            if (typeof result === 'string') {{\n"
            f"                div.appendChild(new DOMParser()\n"
            f"                                    .parseFromString(result, 'text/html')\n"
            f"                                    .body.firstChild);\n"
            f"                element = div;\n"
            f"            }} else {{\n"
            f"                element = result;\n"
            f"            }}\n"
            f"        }} else {{\n"
            f"            let pre = document.createElement('pre');\n"
            f"            pre.textContent = value.toString();\n"
            f"            div.appendChild(pre);\n"
            f"            element = div;\n"
            f"        }}\n"
            f"        return element.innerHTML;\n"
            f"    }} else {{\n"
            f"        return '<pre>' + JSON.stringify(value) + '</pre>';\n"
            f"    }}\n"
            f"}};\n"
        )
        
        # Next, add a workaround for matplotlib.  If you add an html div to your output
        # somewhere, matplotlib's plt.show() will be able to add output to it.
        # Workaround from:
        # https://github.com/iodide-project/pyodide/issues/479
        code += (
            f"window.pyodideLoaded.then(() => {{\n"
            f"    window.iodide = {{\n"
            f"        output: {{\n"
            f"            // Create a new element with tagName and add it to an element with id 'root'\n"
            f"            element: (tagName) => {{\n"
            f"                let elem = document.createElement(tagName);\n"
            f"                document.querySelector('#matplotlib').appendChild(elem);\n"
            f"                return elem;\n"
            f"            }}\n"
            f"        }},\n"
            f"        // Avoid error while loading pyodide.js if this workaround is done too soon\n"
            f"        addOutputRenderer: (opts) => {{}}\n"
            f"    }};\n"
            f"}});\n"
        )
    
    # Placeholder div for output
    div = f"output{_pyodide_count}"
    display(HTML(f"<div id='{div}'></div>"))    

    # Run the code async then replace placeholder if there's non-null output.
    # Note that with runPythonAsync (unlike runPython), packages will be automatically
    # loaded when first imported.
    # PyProxy rendering code from Iodide taken from the bottom of pyodide.js.
    code += (
        f"window.pyodideLoaded.then(() => {{\n"
        f"    var promise = pyodide.runPythonAsync(`\n{cell}\n`, () => {{}});\n"
        f"    promise.then((value) => {{\n"
        f"        if (value == null) {{\n"
        f"            return;\n"
        f"        }} else {{\n"
        f"            document.getElementById('{div}').innerHTML = window.pyodideRender(value);\n"
        f"        }}\n"
        f"    }});\n"
        f"}});\n"
    )

    display(Javascript(code))



## Plain python

Print statements don't show up.  Use `js.console.log` to print to console.

In [ ]:
%%pyodide

from js import console
console.log('hello world')
a = [1, 2, 3, 4, 5]
len(a)

In [ ]:
%%pyodide

d = {'number': 42, 'string': 'hello'}
d

## Pandas

Note you don't have to install pandas; pyodide will install it on the fly you import it!  I'm guessing that only works for [packages they've pre-compiled](https://github.com/iodide-project/pyodide/tree/master/packages).

In [ ]:
%%pyodide

import pandas as pd
df = pd.DataFrame({'AAA': [4, 5, 6, 7],
   'BBB': [10, 20, 30, 40],
   'CCC': [100, 50, -30, -50]})
df

## Matplotlib and widgets

### UI Layout

We have to do this in HTML since ipywidgets would rely on the Jupyter kernel. 

Matplotlib's `plt.show()` doesn't return anything so we have to give it somewhere to go.  We put in an empty `matplotlib` div as a placeholder for the chart; note this name matches the `#matplotlib` in the workaround we set up above.

In [ ]:
%%HTML

<div id='middle'>
    <label for="volume">σ<sub>1</sub></label>
    <input id="sig1" type="range" name="sig" min="1" max="1000">
    <label for="volume">σ<sub>2</sub></label>
    <input id="sig2" type="range" name="sig" min="1" max="1000">
    <label for="volume">σ<sub>3</sub></label>
    <input id="sig3" type="range" name="sig" min="1" max="1000">
</div>

<div id='matplotlib'>
</div>


### Draw the graphs

Note they don't appear in the output area of this cell, they appear in the UI we created above.  Kinda like using the ipywidgets `Output` widget.

This just the initial graph; the sliders don't work yet.

Problem: if you run this more than once, you'll get multiple chart. This is because the matplotlib workaround uses `appendChild` rather than replacing the div contents.  But you might want to show multiple (different) charts, so appending seems like the right answer (?).

In [ ]:
%%pyodide

SIG1 = .75
SIG2 = 1.0
SIG3 = 1.5

import numpy as np
import matplotlib.pyplot as plt

data1 = np.log(np.random.lognormal(1, SIG1, 10000))
data2 = np.log(np.random.lognormal(1, SIG2, 10000))
data3 = np.log(np.random.lognormal(1, SIG3, 10000))
bins = list(np.linspace(np.log(0.01), np.log(100), 100)) + [np.inf]

plt.figure()
plt.title('Log-normal distribution with log bucketing')
plt.hist(data1, bins=bins, histtype='step', label=f"σ={SIG1}")
plt.hist(data2, bins=bins, histtype='step', label=f"σ={SIG2}")
plt.hist(data3, bins=bins, histtype='step', label=f"σ={SIG3}")
plt.legend()
plt.show()

### Connecting the widgets

Now the sliders will change the graph.

In [ ]:
%%pyodide

from js import document

def up1(evt):
    global data1, data2, data3, SIG1
    SIG1 = int(evt.target.value) / 200.
    data1 = np.log(np.random.lognormal(1, SIG1, 10000))
    plt.cla()
    plt.title('Log-normal distribution with log bucketing')
    plt.hist(data1, bins=bins, histtype='step', label=f"σ={SIG1}")
    plt.hist(data2, bins=bins, histtype='step', label=f"σ={SIG2}")
    plt.hist(data3, bins=bins, histtype='step', label=f"σ={SIG3}")
    plt.legend()
    plt.show()

def up2(evt):
    global data1, data2, data3, SIG2
    SIG2 = int(evt.target.value) / 200.
    data2 = np.log(np.random.lognormal(1, SIG2, 10000))
    plt.cla()
    plt.title('Log-normal distribution with log bucketing')
    plt.hist(data1, bins=bins, histtype='step', label=f"σ={SIG1}")
    plt.hist(data2, bins=bins, histtype='step', label=f"σ={SIG2}")
    plt.hist(data3, bins=bins, histtype='step', label=f"σ={SIG3}")
    plt.legend()
    plt.legend()
    plt.show()

def up3(evt):
    global data1, data2, data3, SIG3
    SIG3 = int(evt.target.value) / 200.
    data3 = np.log(np.random.lognormal(1, SIG3, 10000))
    plt.cla()
    plt.title('Log-normal distribution with log bucketing')
    plt.hist(data1, bins=bins, histtype='step', label=f"σ={SIG1}")
    plt.hist(data2, bins=bins, histtype='step', label=f"σ={SIG2}")
    plt.hist(data3, bins=bins, histtype='step', label=f"σ={SIG3}")
    plt.legend()
    plt.show() 
    
document.getElementById('sig1').addEventListener('input', up1)
document.getElementById('sig2').addEventListener('input', up2)
document.getElementById('sig3').addEventListener('input', up3)

## Convert to HTML

Once you run all the cells (don't clear the cell outputs!), you can File -> Download as -> HTML and you have a static version - and the javascript will run your python code *live* when you view the HTML file (watch the console to see things happening).  For some reason matplotlib is erroring out ("no module" error even though pyodide claims to have loaded it) in the HTML version but not the Jupyter version.  No idea why.